In [60]:
import pandas as pd
import numpy as np
import datetime as dt

In [116]:
df_meta = pd.read_csv("TCGAbiolinks_BRCA.csv", header=0, index_col=0, skiprows=0)
df_meta.dropna(how="all", axis=1)
df_meta["birth_year"] = [(dt.datetime.now()+dt.timedelta(days=d)) if d!="" else np.nan for d in df_meta["days_to_birth"].fillna("")]
df_meta["death_year"] = [(dt.datetime.now()-dt.timedelta(days=d)) if d!="" else np.nan for d in df_meta["days_to_death"].fillna("")]
df_meta["age"] = (df_meta["death_year"] - df_meta["birth_year"]).apply(lambda timestamp: timestamp/dt.timedelta(days=365))
bins = np.linspace(df_meta["age"].min(),df_meta["age"].max(), 3)
df_meta["age_binned"] = pd.cut(df_meta["age"], bins=bins, labels = bins[:-1], include_lowest=True)
df_meta.columns

Index(['Tumor.Type', 'Included_in_previous_marker_papers', 'vital_status',
       'days_to_birth', 'days_to_death', 'days_to_last_followup',
       'age_at_initial_pathologic_diagnosis', 'pathologic_stage',
       'Tumor_Grade', 'BRCA_Pathology', 'BRCA_Subtype_PAM50', 'MSI_status',
       'HPV_Status', 'tobacco_smoking_history', 'CNV Clusters',
       'Mutation Clusters', 'DNA.Methylation Clusters', 'mRNA Clusters',
       'miRNA Clusters', 'lncRNA Clusters', 'Protein Clusters',
       'PARADIGM Clusters', 'Pan-Gyn Clusters', 'birth_year', 'death_year',
       'age', 'age_binned'],
      dtype='object')

In [142]:
df_tcgabiolinks = pd.read_csv("../TCGA_biolinks.csv", index_col=0).dropna(how="all", axis=1)
df_tcgabiolinks = df_tcgabiolinks[df_tcgabiolinks["cancer.type"]=="BRCA"].set_index("pan.samplesID")
df_tcgabiolinks["Subtype_Selected_Lum"]=df_tcgabiolinks["Subtype_Selected"]
df_tcgabiolinks["Subtype_Selected_Lum"].replace("BRCA.LumA","BRCA.Lum", inplace=True)
df_tcgabiolinks["Subtype_Selected_Lum"].replace("BRCA.LumB","BRCA.Lum", inplace=True)

In [144]:
df_files=pd.read_csv("files_tcga.dat", index_col=2)
df_files = df_files.reset_index().set_index("cases.0.submitter_id")
df_files = df_files.transpose().append(df_meta.reindex(index=df_files.index).transpose()).transpose()
df_files = df_files.reset_index().set_index("cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id")
df_files = df_files.transpose().append(df_tcgabiolinks.reindex(index=df_files.index).transpose()).transpose()
df_files.fillna("unknown").set_index("file_name").to_csv("files.dat", index=True, header=True)

In [146]:
df_files.groupby(["BRCA_Subtype_PAM50","Subtype_Selected_Lum", "Subtype_Selected"]).count()

cases.0.submitter_id  \
BRCA_Subtype_PAM50 Subtype_Selected_Lum Subtype_Selected                         
Basal              BRCA.Basal           BRCA.Basal                         188   
                   BRCA.Normal          BRCA.Normal                         17   
Her2               BRCA.Her2            BRCA.Her2                           82   
                   BRCA.Normal          BRCA.Normal                          9   
LumA               BRCA.Lum             BRCA.LumA                          570   
                   BRCA.Normal          BRCA.Normal                         53   
LumB               BRCA.Lum             BRCA.LumA                            1   
                                        BRCA.LumB                          209   
                   BRCA.Normal          BRCA.Normal                         21   
Normal             BRCA.Normal          BRCA.Normal                         42   

                                                          cases.0.diagnoses.0.primary_diagnosis  \
BRCA_Subtype_PAM50 Subtype_Selected_Lum Subtype_Selected                                          
Basal              BRCA.Basal           BRCA.Basal                                          188   
                   BRCA.Normal          BRCA.Normal                                          17   
Her2               BRCA.Her2            BRCA.Her2                                            82   
                   BRCA.Normal          BRCA.Normal                                           9   
LumA               BRCA.Lum             BRCA.LumA                                           569   
                   BRCA.Normal          BRCA.Normal                                          53   
LumB               BRCA.Lum             BRCA.LumA                                             1   
                                        BRCA.LumB                                           209   
                   BRCA.Normal          BRCA.Normal                                          21   
Normal             BRCA.Normal          BRCA.Normal                                          42   

                                                          file_name  \
BRCA_Subtype_PAM50 Subtype_Selected_Lum Subtype_Selected              
Basal              BRCA.Basal           BRCA.Basal              188   
                   BRCA.Normal          BRCA.Normal              17   
Her2               BRCA.Her2            BRCA.Her2                82   
                   BRCA.Normal          BRCA.Normal               9   
LumA               BRCA.Lum             BRCA.LumA               570   
                   BRCA.Normal          BRCA.Normal              53   
LumB               BRCA.Lum             BRCA.LumA                 1   
                                        BRCA.LumB               209   
                   BRCA.Normal          BRCA.Normal              21   
Normal             BRCA.Normal          BRCA.Normal              42   

                                                          cases.0.diagnoses.0.morphology  \
BRCA_Subtype_PAM50 Subtype_Selected_Lum Subtype_Selected                                   
Basal              BRCA.Basal           BRCA.Basal                                   188   
                   BRCA.Normal          BRCA.Normal                                   17   
Her2               BRCA.Her2            BRCA.Her2                                     82   
                   BRCA.Normal          BRCA.Normal                                    9   
LumA               BRCA.Lum             BRCA.LumA                                    569   
                   BRCA.Normal          BRCA.Normal                                   53   
LumB               BRCA.Lum             BRCA.LumA                                      1   
                                        BRCA.LumB                                    209   
                   BRCA.Normal          BRCA.Normal                                   21   
Normal             

In [4]:
df=pd.read_csv("mainTable_all.csv", index_col=0, header=0).applymap(lambda x: np.log2(x+1))
df.info()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-b88ac5a1166e>", line 1, in <module>
    df=pd.read_csv("mainTable_all.csv", index_col=0, header=0).applymap(lambda x: np.log2(x+1))
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py", line 6944, in applymap
    return self.apply(infer)
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py", line 6878, in apply
    return op.get_result()
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/apply.py", line 186, in get_result
    return self.apply_standard()
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/apply.py", line 313, in apply_standard
    results, res_index = self.apply_series_generator()
  File "/opt/conda/lib/python3.7/site-packages/pandas/core/apply.py", line 341, in apply_series_generator
    results[i] = self.f(

KeyboardInterrupt: 

In [ ]:
dey = pd.read_csv("http://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None)

In [8]:
df=df.reindex(index=dey.values.ravel()[np.random.randint(0,len(dey), size=1000)])

In [9]:
full_files = [c for c in df_files['file_name'].values if c in df.columns]

In [10]:
df[full_files].dropna().astype(int).to_csv("mainTable.csv")

In [11]:
df_files[df_files['file_name'].isin(full_files)].set_index('file_name').to_csv("files.dat")